In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Import datasets

In [3]:
%cd /content/drive/Shareddrives/CS224N
icd = pd.read_csv(r'diagnosis.csv', dtype="str")
notes = pd.read_csv(r'discharge.csv', dtype="str")

/content/drive/Shareddrives/CS224N


In [ ]:
icd

Print notes

In [ ]:
notes

Get single note

In [6]:
notes.iloc[0].text

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin

Simulate clinical notes using structured EHR data

In [ ]:
"""for index, row in icd.iterrows():
    subject_id = row["subject_id"]
    notes = ""

    # Find gender
    patient = patients.loc[patients["subject_id"] == subject_id]
    if patient.iloc[0, 2] == "M":
      notes += "Patient gender is male "
    else:
      notes += "Patient gender is female "
    
    # Find dob
    notes += "They were born on " + patient.iloc[0, 3] + " "
    
    # Find prescription drugs
    drug_list = drugs.loc[drugs["subject_id"] == subject_id]
    if drug_list is not None:
      notes += " Prescription drugs are "
    for i, r in drug_list.iterrows():
      notes += r["drug"] + " "
    icd.at[index, "notes"] = notes


icd.to_csv('simulated_notes.csv')"""



'for index, row in icd.iterrows():\n    subject_id = row["subject_id"]\n    notes = ""\n\n    # Find gender\n    patient = patients.loc[patients["subject_id"] == subject_id]\n    if patient.iloc[0, 2] == "M":\n      notes += "Patient gender is male "\n    else:\n      notes += "Patient gender is female "\n    \n    # Find dob\n    notes += "They were born on " + patient.iloc[0, 3] + " "\n    \n    # Find prescription drugs\n    drug_list = drugs.loc[drugs["subject_id"] == subject_id]\n    if drug_list is not None:\n      notes += " Prescription drugs are "\n    for i, r in drug_list.iterrows():\n      notes += r["drug"] + " "\n    icd.at[index, "notes"] = notes\n\n\nicd.to_csv(\'simulated_notes.csv\')'

Remove stop words

In [ ]:
"""import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
  
for index, row in icd.iterrows():
  word_tokens = word_tokenize(row["notes"])

  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  
  for w in word_tokens:
    if w not in stop_words:
      filtered_sentence.append(w)
  icd.at[index, "notes"] = filtered_sentence

icd.to_csv('preprocessed_simulated_notes.csv')"""

'import nltk\nnltk.download(\'stopwords\')\nnltk.download(\'punkt\')\n\nstop_words = set(stopwords.words(\'english\'))\n  \nfor index, row in icd.iterrows():\n  word_tokens = word_tokenize(row["notes"])\n\n  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]\n  filtered_sentence = []\n  \n  for w in word_tokens:\n    if w not in stop_words:\n      filtered_sentence.append(w)\n  icd.at[index, "notes"] = filtered_sentence\n\nicd.to_csv(\'preprocessed_simulated_notes.csv\')'

In [7]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import nltk, string, re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
stop_words = set(re.sub(r'[^a-zA-Z\s]', '', stop_word) for stop_word in stopwords.words('english'))
strip_from = "date of birth"

def filter(text):
  text = text.replace('[#,@,&]', '')
  text = text.replace('\d*','')
  text = text.replace('w{3}','')
  text = text.replace("http\S+", "")
  text = text.replace('\s+', '')
  text = re.sub("\S*\d\S*", "", text)
  text = text.replace(r'\s+[a-zA-Z]\s+', '').lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text_pos = text.find(strip_from) + len(strip_from) # Strip useless prefix discharge summary info
  text = text[text_pos:]
  text = ' '.join([word for word in text.split() if word not in (stop_words)])
  return text

Create input for models

In [ ]:
from numpy import NaN
from tables.description import IsDescription

#obesity_icds = ['278', '493', '414', '428', '296', '250', '560', '530', '274', 
#                '272', '401', '272', '32723', '7151', '4439', '45981']
conditions = ["obesity", "diabetes", "hypercholesterolemia", 
              "hypertension", "atherosclerotic",
              "heart failure", "peripheral vascular disease",
              "venous insufficiency", "osteoarthritis", 
              "sleep apnea", "asthma", "reflux", "gallstone",
              "depression", "gout"]

columns = ["text"] + conditions
input = pd.DataFrame(index=range(len(pd.unique(notes['subject_id']))), columns=columns)
# 278 maps to obesity diagnoses in ICD-9
ids = []

for index, row in notes.iterrows():
  subject_id = row.subject_id
  
  if subject_id not in ids:
    summary = filter(row.text)
    ind = len(ids)
    input.at[ind, "text"] = summary
    ids.append(subject_id)
    diagnoses_rows = icd[icd.subject_id == subject_id]
    for j in range(len(conditions)):
      input.at[ind, conditions[j]] = 0

    for i in range(len(diagnoses_rows)):
      description = diagnoses_rows.iloc[i].icd_title.lower()
      for j in range(len(conditions)):    
        if conditions[j] in description:
          input.at[ind, conditions[j]] = 1


input = input.dropna()
input.to_csv('formatted_model_input1000.csv', index=False)


In [ ]:
input